### This is a training file

In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import pickle
from sklearn.metrics.pairwise import cosine_similarity
#from ingredient_parser import ingredient_parser
import re
import pickle
 

In [2]:
# Dataset loading and saving as Dataframe
# !unzip Cleaned_Indian_Food_Dataset.csv.zip
df = pd.read_csv('Cleaned_Indian_Food_Dataset.csv')

### TF_IDF 

In [3]:
tfidf = TfidfVectorizer()
tfidf.fit(df['Cleaned-Ingredients']) #vectorizer Object
tfidf_r = tfidf.fit_transform(df['Cleaned-Ingredients']) # sparse matrix of tf-idf values of the vocab
                                                        # in the raw document


In [4]:
print(f'Unique Features (words) from the Ingredients are : {len(tfidf_r.toarray()[0])}')
print(f'Number of rows in the Dataframe : {len(tfidf_r.toarray())}')

Unique Features (words) from the Ingredients are : 2385
Number of rows in the Dataframe : 5938


In [15]:
# save vectorizer

with open("tfidf_vec.pkl", "wb") as f:
  pickle.dump(tfidf, f)


In [16]:
# save tfidf_encoder

with open("tfidf_en.pkl", "wb") as f:
  pickle.dump(tfidf_r, f)

### User Input

In [8]:
user_ing = input()
ing_user = tfidf.transform([user_ing])

scoring = cosine_similarity(ing_user, tfidf_r) # second param gets transposed for dot calculation
scoring = list(scoring[0]) # given it has only one input each session

In [9]:

max_idx = scoring.index(max(scoring))

print(f'Highest :{max(scoring)}, index is {max_idx}')

Highest :0.5143193693875535, index is 1528


In [10]:
sorted(scoring, reverse=True)[:3] # reverse does descending order


[0.5143193693875535, 0.430269972309515, 0.3942177331432274]

In [ ]:
user_ing = input("What's in your fridge? ")

What's in your fridge? chicken, rice


## Recommender

In [18]:
with open('tfidf_vec.pkl', 'rb') as f:
  tfidf_vec = pickle.load(f)

with open('tfidf_en.pkl', 'rb') as f:
  tfidf_en = pickle.load(f)

In [19]:
def get_cos_sim(user_input_encoded, tfidf_encoded): # returns scores
  scores = cosine_similarity(user_input_encoded, tfidf_encoded)
  scores = list(scores[0])
  return scores

In [20]:
def get_top_scores(scores, top_N): # returns sorted N scores as list

  top_sorted_scores = sorted(scores, reverse=True)[:top_N]

  idx_list = [scores.index(i) for i in top_sorted_scores] # get list of indexes with high scores
  return idx_list


In [21]:
def get_recommendation(df, top_N):

  user_ing = input("What's in your fridge? ")
  ing_user = tfidf_vec.transform([user_ing])
  scores = get_cos_sim(ing_user, tfidf_en)
  top_idx_list = get_top_scores(scores, top_N)
  recommendations = [ df['TranslatedRecipeName'][i] for i in top_idx_list ]  
  rec_instructions = [ df['TranslatedInstructions'][i] for i in top_idx_list ]
  rec_ingredients = [ df['TranslatedIngredients'][i] for i in top_idx_list ]
  return recommendations, rec_instructions, rec_ingredients

In [22]:
recommendations, rec_instructions, rec_ingredients = get_recommendation(df, 3)

What's in your fridge? chicken, rice


In [23]:
recommendations

['Chicken In Tomato Onion Gravy Recipe',
 'Dragon Chicken Sizzler Recipe',
 'Sel Roti Recipe (Nepalese ring-Shaped Sweet bread)']

In [24]:
print(f'You can try recipe(s): {recommendations}')

You can try recipe(s): ['Chicken In Tomato Onion Gravy Recipe', 'Dragon Chicken Sizzler Recipe', 'Sel Roti Recipe (Nepalese ring-Shaped Sweet bread)']


In [25]:
rec_instructions[0]

'To begin making Chicken In Tomato Onion Gravy Recipe, take onions, green chillies and tomatoes (de seeded) in a mixer jar and grind to a smooth paste.In a pressure cooker, add the onion-chilli and tomato paste, curd, salt, chicken masala powder, lemon juice and ginger garlic paste.\nStir to combine.Add the chicken pieces and about half cup of water and close the lid of the pressure cooker and cook on high flame for about 3 whistles or till the chicken is cooked till tender.Once the pressure is released on its own, heat a bit of oil in a kadai and once the oil is hot, transfer the gravy of the cooked chicken from the pressure cooker.\nRetain the cooked chicken in the pressure cooker.\nyou may do this by refraining the chicken with a slotted spoon while pouring the gravy.Cook for a few minutes while stirring, till the gravy starts bubbling and thickening.Once the gravy is thickened to desired consistency, add the chicken pieces to the kadai, close a lid and cook on a low flame for about